# Tutorial for setting up and simulating dsDNA with MRG-CG DNA model

In [9]:
# load packages
import numpy as np
import pandas as pd
import sys
import os
import argparse
import simtk.openmm as mm
import simtk.openmm.app as app
import simtk.unit as unit

sys.path.append('../../')
from openabc.forcefields.parsers import MRGdsDNAParser
from openabc.forcefields import MOFFMRGModel

We simulate a 200 bp dsDNA by MRG-CG DNA model. Similar to MOFF, for dsDNA, we also provide a simple way to parse DNA topology and get all the bonds, angles, fan bonds, etc. Use class `MRGdsDNAParser` to parse each dsDNA. The nonbonded exclusion list includes CG DNA atoms involved in bonds or angles, but those involved in fan bonds are not included. It is important that the parser cannot automatically recognize which nucleotides should be paired, so the input has to be the atomistic model of a single dsDNA with correct nucleotide index so that nucleotide i is paired with nucleotide N - i - 1 (nucleotide index starts from 0 and N is the total number of nucleotides in dsDNA). 

Also, as the original MRG-CG DNA model is designed with explicit ions, to apply this model with implicit ion, by default we scale all the bonded interaction (bond, angle, and fan bond) force constants by 0.9 to keep correct thermodynamic properties such as the persistence length. 

In [10]:
# parse dsDNA with atomistic model as input
dsDNA = MRGdsDNAParser.from_atomistic_pdb('input-pdb/all_atom_200bpDNA.pdb', 'MRG_dsDNA.pdb')

Parse molecule with default settings.


Let's take a look at the bonds, angles, and fan bonds. You can see the force constants are scaled. 

In [11]:
print(dsDNA.dna_bonds.head())
print(dsDNA.dna_angles.head())
print(dsDNA.dna_fan_bonds.head())

    a1   a2     r0  k_bond_2  k_bond_3  k_bond_4
0  0.0  1.0  0.496    988.47 -851.0256  561.0744
1  1.0  2.0  0.496    988.47 -851.0256  561.0744
2  2.0  3.0  0.496    988.47 -851.0256  561.0744
3  3.0  4.0  0.496    988.47 -851.0256  561.0744
4  4.0  5.0  0.496    988.47 -851.0256  561.0744
    a1   a2   a3    theta0  k_angle_2  k_angle_3  k_angle_4
0  0.0  1.0  2.0  2.722714  34.718832  15.664896   4.059317
1  1.0  2.0  3.0  2.722714  34.718832  15.664896   4.059317
2  2.0  3.0  4.0  2.722714  34.718832  15.664896   4.059317
3  3.0  4.0  5.0  2.722714  34.718832  15.664896   4.059317
4  4.0  5.0  6.0  2.722714  34.718832  15.664896   4.059317
    a1     a2     r0   k_bond_2   k_bond_3    k_bond_4
0  0.0  394.0  1.710  17.585352    7.90776    5.497776
1  0.0  395.0  1.635   0.000499  -45.94032   69.663600
2  0.0  396.0  1.470  32.007600 -167.19264  188.280000
3  0.0  397.0  1.345  46.316880 -150.62400  139.327200
4  0.0  398.0  1.230  15.062400  -37.65600   30.124800


Now we can build the system and run simulation. We first build an instance of `MOFFMRGModel`, then we append the dsDNA parser instance into it and set up the simulation. 

In [12]:
dna = MOFFMRGModel()
dna.append_mol(dsDNA)
top = app.PDBFile('MRG_dsDNA.pdb').getTopology()
dna.create_system(top)
salt_conc = 100*unit.millimolar
temperature = 300*unit.kelvin
dna.add_dna_bonds(force_group=1)
dna.add_dna_angles(force_group=2)
dna.add_dna_fan_bonds(force_group=3)
dna.add_contacts(force_group=4)
dna.add_elec_switch(salt_conc, temperature, force_group=5) # electrostatic interaction depends on salt concentration and temperature
dna.save_system('dsDNA.xml')
collision = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.NoseHooverIntegrator(temperature, collision, timestep)
platform_name = 'CPU'
init_coord = app.PDBFile('MRG_dsDNA.pdb').getPositions()
dna.set_simulation(integrator, platform_name, init_coord=init_coord)
dna.simulation.minimizeEnergy()
output_interval = 100
output_dcd = 'output.dcd'
dna.add_reporters(output_interval, output_dcd)
dna.simulation.context.setVelocitiesToTemperature(temperature)
dna.simulation.step(500)


Add DNA bonds.
Add DNA angles.
Add DNA fan bonds.
Add protein and DNA nonbonded contacts.
Add protein and DNA electrostatic interactions with distance-dependent dielectric and switch.
Do not add electrostatic interactions between native pair atoms.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,1.0000000000000007,1879.2325945371576,624.4567708800065,2503.689365417164,125.48853209041671,0
200,2.0000000000000013,1633.8708673976262,1144.651053769159,2778.5219211667854,230.02485871170398,88.4
300,2.99999999999998,1750.732864585787,1347.9887853197035,3098.7216499054903,270.88686011960647,88
400,3.9999999999999587,2115.820925045079,1310.21345440284,3426.034379447919,263.2956687881286,89.7
500,4.999999999999938,2173.3021696193746,1526.5531857062795,3699.8553553256543,306.77050416519666,86.6
